# Conversational Movie Recommendations with RecBole and Gemini

**Objective:** This notebook demonstrates how a pre-trained RecBole recommendation model can be used for inference to provide personalized movie recommendations within a simulated chatbot conversation powered by the Gemini API. The chatbot will also attempt to provide explanations for its recommendations based on the user's (simulated) viewing history.

**Steps:**
1.  **Setup & Configuration:** Import libraries, load API keys, configure SDKs, define paths, and set parameters (e.g., which RecBole model to load).
2.  **Load MovieLens Item Data:** Load `u.item` to get movie titles and genres.
3.  **Load Pre-trained RecBole Model & Dataset:** Load a saved model checkpoint and the corresponding dataset object from the previous RecBole evaluation notebook.
4.  **Simulate User Profile & History:** Define a sample user and their liked movies.
5.  **Generate Recommendations:** Use the loaded RecBole model to get recommendations for the simulated user.
6.  **Conversational Agent (Gemini):**
    a.  Define a function to interact with the Gemini API.
    b.  Craft a prompt that takes the user's request, recommended movies (with genres), and liked movies (with genres) to generate a conversational response with explanations.
7.  **Showcase Interaction:** Run an example chat interaction.

**CRITICAL PREREQUISITES:**
- Ensure libraries are installed:
  `pip install recbole pandas python-dotenv google-generativeai`
- A `.env` file in the project root with `GEMINI_API_KEY`.
- A RecBole model checkpoint and dataset files saved from `04_recbole_offline_evaluation_v2.ipynb` in the `recbole_saved_models` and `recbole_data` directories respectively.
- **The `u.item` file from the MovieLens 100k dataset MUST BE PRESENT in the `[PROJECT_ROOT]/recbole_data/ml-100k/` directory.**
    - RecBole (in notebook `04`) should download and place this file here when it first processes the `ml-100k` dataset.
    - **If `u.item` is NOT found at this location after running notebook `04` successfully, you MUST manually download the `ml-100k.zip` file from [https://grouplens.org/datasets/movielens/100k/](https://grouplens.org/datasets/movielens/100k/), extract it, and copy `u.item` (and preferably other `u.*` files) into your `[PROJECT_ROOT]/recbole_data/ml-100k/` directory.** This notebook cannot function without `u.item` for movie titles and genres.
- **If the chosen RecBole model (e.g., LightGCN) required manual patches to its source code in the previous notebook, those patches must still be in effect in your RecBole installation for this notebook to work correctly.**


In [5]:
import os
import json
import pandas as pd
from dotenv import load_dotenv
import asyncio
import random
import torch # Explicitly import torch for device checking if needed

# For running async code in Jupyter environments smoothly
import nest_asyncio
nest_asyncio.apply()

# Google Generative AI SDK
import google.generativeai as genai
from google.generativeai.types import HarmCategory, HarmBlockThreshold

# RecBole imports for loading model and data
from recbole.quick_start import load_data_and_model
# from recbole.utils.case_study import full_sort_scores, full_sort_topk # Not directly used, model.full_sort_predict is used
from recbole.data.interaction import Interaction

print("Libraries imported successfully.")

Libraries imported successfully.


## 2. Configuration

Define essential variables for API keys, paths, the RecBole model to load, and other parameters.

In [6]:
def load_api_key(project_r):
    env_path = os.path.join(project_r, '.env')
    if os.path.exists(env_path):
        load_dotenv(dotenv_path=env_path)
        print(f".env file loaded from: {env_path}")
    else:
        load_dotenv()
        if os.path.exists(".env"): print(f".env file loaded from current directory: {os.getcwd()}/.env")
        else: print(f"Warning: .env file not found at {env_path} or in current directory.")
    api_key_loaded = os.getenv("GEMINI_API_KEY")
    if not api_key_loaded: print("Warning: GEMINI_API_KEY not found.")
    else: print("GEMINI_API_KEY loaded.")
    return api_key_loaded

# --- Determine Project Root ---
current_working_dir = os.getcwd()
print(f"Current working directory (os.getcwd()): {current_working_dir}")
if os.path.basename(current_working_dir).lower() == "notebooks":
    PROJECT_ROOT = os.path.abspath(os.path.join(current_working_dir, ".."))
else:
    PROJECT_ROOT = current_working_dir 
print(f"PROJECT_ROOT set to: {PROJECT_ROOT}")

API_KEY = load_api_key(PROJECT_ROOT)

# --- Configure Google Generative AI SDK ---
SDK_CONFIGURED_SUCCESSFULLY = False
if API_KEY: 
    try:
        genai.configure(api_key=API_KEY)
        SDK_CONFIGURED_SUCCESSFULLY = True
        print("Google Generative AI SDK configured successfully.")
    except Exception as e: print(f"Error configuring Google Generative AI SDK: {e}")
else: print("Google Generative AI SDK not configured due to missing API key.")

# --- Directory and File Paths ---
DATA_PATH = os.path.join(PROJECT_ROOT, "recbole_data")
SAVED_MODELS_PATH = os.path.join(PROJECT_ROOT, "recbole_saved_models")
ML_100K_RAW_PATH = os.path.join(DATA_PATH, "ml-100k") # Path where RecBole downloads ml-100k

# --- Check for u.item (CRITICAL PREREQUISITE) ---
U_ITEM_EXPECTED_PATH = os.path.join(ML_100K_RAW_PATH, 'u.item')
if not os.path.exists(U_ITEM_EXPECTED_PATH):
    print(f"!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
    print(f"CRITICAL ERROR: MovieLens item data file 'u.item' not found at: {U_ITEM_EXPECTED_PATH}")
    print(f"This file is essential for mapping item IDs to titles and genres.")
    print(f"It is typically downloaded by RecBole when you run notebook '04_recbole_offline_evaluation_v2.ipynb'.")
    print(f"If it's missing, please MANUALLY download ml-100k.zip from https://grouplens.org/datasets/movielens/100k/,")
    print(f"extract it, and copy 'u.item' into '{ML_100K_RAW_PATH}'.")
    print(f"Stopping further execution in this cell as 'u.item' is missing.")
    print(f"!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
    U_ITEM_FOUND = False
else:
    U_ITEM_FOUND = True
    print(f"'u.item' file found at: {U_ITEM_EXPECTED_PATH}")


# --- RecBole Model Loading Configuration ---
MODEL_TO_LOAD = 'LightGCN' # The model type we intend to load

# User-provided path for the specific model checkpoint
USER_SPECIFIED_MODEL_SUBPATH = "LightGCN_20250520_213658/LightGCN-May-20-2025_21-38-20.pth"
MODEL_CHECKPOINT_PATH = os.path.join(SAVED_MODELS_PATH, "ml-100k", USER_SPECIFIED_MODEL_SUBPATH)

print(f"Attempting to load specified model checkpoint: {MODEL_CHECKPOINT_PATH}")

if not os.path.exists(MODEL_CHECKPOINT_PATH):
    print(f"CRITICAL: Specified model checkpoint file not found at '{MODEL_CHECKPOINT_PATH}'.")
    print("Please ensure the path is correct and the model file exists from the previous notebook's run.")
    # Fallback to auto-detection if user-specified path is not found (optional, can be removed if direct path is mandatory)
    print("Attempting auto-detection as a fallback...")
    MODEL_CHECKPOINT_DIR_BASE = os.path.join(SAVED_MODELS_PATH, "ml-100k")
    MODEL_CHECKPOINT_PATH = None # Reset for auto-detection
    if os.path.exists(MODEL_CHECKPOINT_DIR_BASE):
        model_dirs = [d for d in os.listdir(MODEL_CHECKPOINT_DIR_BASE) if os.path.isdir(os.path.join(MODEL_CHECKPOINT_DIR_BASE, d)) and d.startswith(MODEL_TO_LOAD)]
        if model_dirs:
            latest_model_dir_name = sorted(model_dirs, reverse=True)[0] 
            potential_model_dir = os.path.join(MODEL_CHECKPOINT_DIR_BASE, latest_model_dir_name)
            pth_files = [f for f in os.listdir(potential_model_dir) if f.endswith(".pth")]
            if pth_files:
                MODEL_CHECKPOINT_PATH = os.path.join(potential_model_dir, pth_files[0])
                print(f"Auto-detected model checkpoint to load: {MODEL_CHECKPOINT_PATH}")
            else:
                print(f"Warning (auto-detection): No .pth file found in directory {potential_model_dir} for model {MODEL_TO_LOAD}.")
        else:
            print(f"Warning (auto-detection): No directories found starting with '{MODEL_TO_LOAD}' in {MODEL_CHECKPOINT_DIR_BASE}.")
    else:
        print(f"Warning (auto-detection): Base model checkpoint directory not found: {MODEL_CHECKPOINT_DIR_BASE}")

    if MODEL_CHECKPOINT_PATH is None:
         print(f"CRITICAL (auto-detection): Could not automatically find a checkpoint for model '{MODEL_TO_LOAD}'.")

# --- Other Parameters ---
NUM_RECOMMENDATIONS = 5 # Number of movies to recommend

print(f"RecBole data path: {DATA_PATH}")
print(f"MovieLens 100k raw path (expected for u.item): {ML_100K_RAW_PATH}")

Current working directory (os.getcwd()): /mnt/c/Users/tduricic/Development/workspace/conversational-reco/notebooks
PROJECT_ROOT set to: /mnt/c/Users/tduricic/Development/workspace/conversational-reco
.env file loaded from: /mnt/c/Users/tduricic/Development/workspace/conversational-reco/.env
GEMINI_API_KEY loaded.
Google Generative AI SDK configured successfully.
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
CRITICAL ERROR: MovieLens item data file 'u.item' not found at: /mnt/c/Users/tduricic/Development/workspace/conversational-reco/recbole_data/ml-100k/u.item
This file is essential for mapping item IDs to titles and genres.
It is typically downloaded by RecBole when you run notebook '04_recbole_offline_evaluation_v2.ipynb'.
If it's missing, please MANUALLY download ml-100k.zip from https://grouplens.org/datasets/movielens/100k/,
extract it, and copy 'u.item' into '/mnt/c/Users/tduricic/Development/workspace/conversational-reco/recbole_data/ml-100

## 3. Load MovieLens Item Data (Titles and Genres)

We need to load the `u.item` file from the MovieLens 100k dataset to map item IDs to their actual titles and genres. This information is crucial for presenting recommendations to the user and for providing context to the LLM for generating explanations.
This cell will only proceed if `u.item` was found in the previous cell.

In [4]:
def load_movie_titles_and_genres(ml_100k_path):
    """Loads movie titles and genres from u.item."""
    u_item_file = os.path.join(ml_100k_path, 'u.item')
    # The check for u_item_file existence is now primarily in CELL 4.
    # This function assumes the path is valid if called.
    if not os.path.exists(u_item_file): # Defensive check still good
        print(f"Error: u.item file not found at {u_item_file} (should have been caught earlier).")
        return None

    item_cols = [
        'movie_id', 'movie_title', 'release_date', 'video_release_date', 'imdb_url',
        'genre_unknown', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy',
        'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror',
        'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'
    ]
    try:
        movies_df = pd.read_csv(u_item_file, sep='|', header=None, names=item_cols, encoding='latin-1')
        
        genre_columns = item_cols[5:] 
        def get_genres(row):
            active_genres = [col for col in genre_columns if row[col] == 1]
            return ", ".join(active_genres) if active_genres else "N/A"
        
        movies_df['genres_str'] = movies_df.apply(get_genres, axis=1)
        
        movie_info_map = movies_df.set_index('movie_id')[['movie_title', 'genres_str']].apply(
            lambda x: {'title': x['movie_title'], 'genres': x['genres_str']}, axis=1
        ).to_dict()
        
        print(f"Loaded information for {len(movie_info_map)} movies from u.item.")
        return movie_info_map
    except Exception as e:
        print(f"Error loading or processing u.item: {e}")
        return None

movie_info_map = None
if U_ITEM_FOUND: # Only attempt to load if the file was confirmed to exist
    movie_info_map = load_movie_titles_and_genres(ML_100K_RAW_PATH)
    if movie_info_map:
        sample_movie_id = list(movie_info_map.keys())[0]
        print(f"\nSample movie info (ID: {sample_movie_id}): {movie_info_map[sample_movie_id]}")
else:
    print("\nSkipping loading of u.item because the file was not found in the expected location.")

Error: u.item file not found at /mnt/c/Users/tduricic/Development/workspace/conversational-reco/recbole_data/ml-100k/u.item
Please ensure MovieLens 100k data is downloaded by RecBole or placed manually.


## 4. Load Pre-trained RecBole Model and Dataset

This step loads the saved RecBole model checkpoint and its corresponding dataset object.
The dataset object is essential as it contains the mappings between original user/item IDs and RecBole's internal numerical IDs.

**Note:** This cell will fail if `MODEL_CHECKPOINT_PATH` is not correctly set or if the model/dataset files are missing.
If the model (e.g., LightGCN) required manual patching of RecBole's source code to run correctly during training, those patches must still be in effect in your current RecBole installation.


In [ ]:
recbole_model = None
recbole_dataset = None

if MODEL_CHECKPOINT_PATH and os.path.exists(MODEL_CHECKPOINT_PATH) and U_ITEM_FOUND: # Added U_ITEM_FOUND check
    try:
        print(f"Loading RecBole model and dataset from checkpoint: {MODEL_CHECKPOINT_PATH}...")
        config, model, dataset, train_data, valid_data, test_data = load_data_and_model(
            model_file=MODEL_CHECKPOINT_PATH,
        )
        recbole_model = model
        recbole_dataset = dataset
        print(f"Successfully loaded model: {recbole_model.model_name}")
        print(f"Dataset '{recbole_dataset.dataset_name}' loaded with {recbole_dataset.user_num} users and {recbole_dataset.item_num} items.")
        
        recbole_model.eval()
        recbole_model.to(torch.device('cpu')) 
        print(f"Model '{recbole_model.model_name}' is on device: {next(recbole_model.parameters()).device}")

    except Exception as e:
        print(f"Error loading RecBole model or dataset: {e}")
        print("Please ensure the MODEL_CHECKPOINT_PATH is correct and all necessary files (.pth, .dataset) are present.")
        print("Also, ensure your RecBole environment and any manual patches are consistent with the training environment.")
        import traceback
        traceback.print_exc()
elif not U_ITEM_FOUND:
    print("Skipping RecBole model loading because 'u.item' was not found (critical for movie info).")
else:
    print("MODEL_CHECKPOINT_PATH is not set or file does not exist. Cannot load RecBole model.")

## 5. Simulate User Profile & Generate Recommendations

Here, we'll:
1.  Define a sample user and a few movies they have "liked" (using movie titles).
2.  Convert these liked movie titles into RecBole's internal item IDs.
3.  Create an interaction object that RecBole can use for prediction.
4.  Use the loaded RecBole model to generate top-N recommendations for this user.
5.  Map the recommended internal item IDs back to movie titles and genres.

In [ ]:
def get_recommendations_for_user(model, dataset, user_original_id, user_liked_original_item_ids, top_k, movie_info):
    """
    Generates top-K recommendations for a given user and their liked items.
    """
    if model is None or dataset is None:
        print("RecBole model or dataset not loaded. Cannot generate recommendations.")
        return [], []
    if not movie_info: # Check if movie_info_map is available
        print("Movie info (titles/genres) not loaded. Cannot provide full recommendation details.")
        return [], []


    try:
        internal_user_id_tensor = dataset.token2id(dataset.uid_field, [str(user_original_id)])
        
        if internal_user_id_tensor.size == 0 or internal_user_id_tensor[0] == 0: 
             print(f"Warning: User ID '{user_original_id}' not found in dataset's known users or maps to unknown. Using first valid user.")
             first_original_uid = dataset.id2token(dataset.uid_field, [1])[0] 
             internal_user_id_tensor = dataset.token2id(dataset.uid_field, [first_original_uid])
             print(f"Using fallback user: original ID '{first_original_uid}', internal ID {internal_user_id_tensor[0]}")

        user_interaction = Interaction({dataset.uid_field: internal_user_id_tensor})
        user_interaction = dataset.join(user_interaction) 
        user_interaction = user_interaction.to(model.device)

        scores = model.full_sort_predict(user_interaction) 

        internal_liked_ids = []
        if user_liked_original_item_ids:
            internal_liked_ids_tensor = dataset.token2id(dataset.iid_field, [str(iid) for iid in user_liked_original_item_ids])
            internal_liked_ids = [iid.item() for iid in internal_liked_ids_tensor if iid.item() != 0] 

        if internal_liked_ids:
            valid_internal_liked_ids = [iid for iid in internal_liked_ids if iid < scores.shape[1]]
            if valid_internal_liked_ids:
                 scores[0, valid_internal_liked_ids] = -torch.inf 

        top_k_scores, top_k_indices = torch.topk(scores, k=top_k, dim=1)
        
        recommended_internal_ids = top_k_indices.squeeze().tolist()
        if not isinstance(recommended_internal_ids, list): # Handle single recommendation case
            recommended_internal_ids = [recommended_internal_ids]
        
        recommended_movies_details = []
        if movie_info: 
            original_item_ids_str = dataset.id2token(dataset.iid_field, recommended_internal_ids)
            for original_id_str in original_item_ids_str:
                try:
                    original_id_int = int(original_id_str)
                    info = movie_info.get(original_id_int)
                    if info:
                        recommended_movies_details.append({
                            "title": info['title'],
                            "genres": info['genres'],
                            "original_id": original_id_int
                        })
                    else:
                        recommended_movies_details.append({
                            "title": f"Unknown Movie (ID: {original_id_str})",
                            "genres": "N/A",
                            "original_id": original_id_str
                        })
                except ValueError:
                     recommended_movies_details.append({
                            "title": f"Unknown Movie (ID: {original_id_str} - non-integer)",
                            "genres": "N/A",
                            "original_id": original_id_str
                        })

        liked_movies_details = []
        if movie_info and user_liked_original_item_ids:
            for original_id_str in user_liked_original_item_ids: # Iterate through the list of strings
                try:
                    original_id = int(original_id_str) # Convert to int for lookup
                    info = movie_info.get(original_id) 
                    if info:
                        liked_movies_details.append({
                            "title": info['title'],
                            "genres": info['genres'],
                            "original_id": original_id
                        })
                except ValueError:
                    print(f"Warning: Could not convert liked movie ID '{original_id_str}' to integer.")

        
        return recommended_movies_details, liked_movies_details

    except Exception as e:
        print(f"Error getting recommendations: {e}")
        import traceback
        traceback.print_exc()
        return [], []

# --- Simulate a User and their History ---
SIMULATED_USER_ID = "1" 
SIMULATED_LIKED_MOVIE_ORIGINAL_IDS = ["50", "100", "181"] 

recommendations = [] # Initialize to ensure it exists
liked_movie_details_for_prompt = [] # Initialize

if recbole_model and recbole_dataset and movie_info_map: # Ensure movie_info_map is loaded
    recommendations, liked_movie_details_for_prompt = get_recommendations_for_user(
        recbole_model, 
        recbole_dataset, 
        SIMULATED_USER_ID, 
        SIMULATED_LIKED_MOVIE_ORIGINAL_IDS, 
        NUM_RECOMMENDATIONS,
        movie_info_map
    )
    
    print(f"\n--- Simulated User {SIMULATED_USER_ID} ---")
    print("Liked Movies (for prompt context):")
    if liked_movie_details_for_prompt:
        for movie in liked_movie_details_for_prompt:
            print(f"- {movie['title']} (Genres: {movie['genres']})")
    else:
        print("No liked movie details to display (movie_info_map might be missing or IDs not found).")


    print("\nTop Recommendations:")
    if recommendations:
        for i, movie in enumerate(recommendations):
            print(f"{i+1}. {movie['title']} (Genres: {movie['genres']})")
    else:
        print("No recommendations generated or an error occurred.")
elif not movie_info_map:
    print("Movie info (u.item) not loaded. Cannot proceed with user simulation and recommendations.")
else:
    print("RecBole model/dataset not loaded. Skipping recommendation generation.")

## 6. Conversational Agent with Gemini

This section defines a function to interact with the Gemini API. It will take the user's request, the generated movie recommendations (with titles and genres), and the user's liked movies (with titles and genres) to craft a conversational response that includes explanations.

In [ ]:
async def get_conversational_recommendation_with_explanation(user_query, recommended_movies, liked_movies_history):
    if not SDK_CONFIGURED_SUCCESSFULLY:
        return "Sorry, I'm having trouble connecting to my brain right now. Please try again later."
    if not recommended_movies:
        return "I couldn't find any specific recommendations for you right now, but I'm always learning! Perhaps try a broader query?"

    liked_movies_str = ""
    if liked_movies_history:
        liked_movies_parts = [f"'{movie['title']}' (Genres: {movie['genres']})" for movie in liked_movies_history]
        if liked_movies_parts:
            liked_movies_str = "You previously liked: " + ", ".join(liked_movies_parts) + "."

    recs_str_parts = []
    for i, movie in enumerate(recommended_movies):
        recs_str_parts.append(f"{i+1}. '{movie['title']}' (Genres: {movie['genres']})")
    recs_str = "\n".join(recs_str_parts)

    prompt = f"""
    You are a friendly and helpful movie recommendation chatbot.
    A user has asked: "{user_query}"
    {liked_movies_str}

    Here are some movie recommendations for the user:
    {recs_str}

    Your task is to:
    1.  Present these recommendations in a conversational and engaging way.
    2.  For each recommended movie, try to provide a brief, plausible explanation for why the user might like it, ideally by connecting it to their liked movies (considering titles and genres). For example, if they liked a sci-fi movie, and you recommend another sci-fi movie, mention that. If they liked a comedy, and you recommend another, highlight that.
    3.  If you cannot find a strong direct link, provide a more general positive statement about the recommended movie.
    4.  Keep the tone light and friendly. Do not invent movies or facts not provided.
    5.  Structure your response as a single chat message.
    """

    safety_settings = {
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
    }
    model = genai.GenerativeModel(model_name='gemini-1.5-flash-latest', safety_settings=safety_settings)
    
    try:
        response = await model.generate_content_async(contents=[prompt])
        return response.text
    except Exception as e:
        print(f"Error interacting with Gemini API: {e}")
        return "I'm sorry, I encountered an issue while trying to generate your recommendations with explanations. Please try again."

## 7. Showcase Chatbot Interaction

Let's simulate a user asking for recommendations and see how our Gemini-powered chatbot responds, using the recommendations generated by the RecBole model.


In [ ]:
async def run_chatbot_example():
    if not recommendations : # Check if recommendations list is populated and not empty
        print("No recommendations available to run the chatbot example.")
        print("This might be due to errors in loading the RecBole model or generating recommendations.")
        return
    if not liked_movie_details_for_prompt: # Also check liked movies for context
        print("No liked movie details available for context in chatbot example.")
        # Optionally, decide if you want to proceed without liked movie context or stop
        # For now, we'll let it proceed but the LLM won't have liked movies to refer to.

    user_chat_query = "Can you recommend some movies for me?"
    print(f"\nSimulated User Query: {user_chat_query}")
    
    chatbot_response = await get_conversational_recommendation_with_explanation(
        user_chat_query,
        recommendations,
        liked_movie_details_for_prompt 
    )
    
    print("\nChatbot Response:")
    print(chatbot_response)

if SDK_CONFIGURED_SUCCESSFULLY and recbole_model and recbole_dataset and movie_info_map and U_ITEM_FOUND:
    if recommendations: # Ensure recommendations were actually generated
        await run_chatbot_example() 
    else:
        print("\nSkipping chatbot example because no recommendations were generated (check previous steps).")
else:
    print("\nSkipping chatbot example because prerequisites (SDK, model, data, u.item, or initial recommendations) are missing.")

## 8. Discussion on Explanation Quality and Further Improvements

**Explanation Quality:**
- The quality of explanations generated by the LLM heavily depends on:
    - **Prompt Engineering:** How well the prompt guides the LLM to connect liked items with recommendations.
    - **Available Information:** Providing genres (as we did) is helpful. More detailed item metadata (e.g., keywords, plot summaries, actors, directors) for both liked and recommended items would allow for richer and more accurate explanations.
    - **LLM Capabilities:** The LLM's ability to infer relationships and articulate them naturally.
- Simple genre-matching is a good starting point. More sophisticated explanations might require the LLM to understand deeper thematic connections or stylistic similarities.

**Further Improvements:**
1.  **Richer Item Metadata:** Incorporate more detailed movie metadata (plot keywords, director, main actors) into the prompt for both liked and recommended items to enable more nuanced explanations.
2.  **User Feedback Loop:** In a real system, user feedback on recommendations and explanations could be used to fine-tune the LLM prompts or even the recommendation model.
3.  **Multi-Turn Conversation:** Extend the chatbot to handle follow-up questions like "Tell me more about 'Movie X'" or "Recommend something similar but less action-packed."
4.  **Negative Feedback:** Allow users to say they disliked a recommendation, and use that to refine future suggestions and explanations.
5.  **Diversity in Explanations:** Prompt the LLM to vary its explanation styles to avoid sounding repetitive.
6.  **Knowledge Cutoff:** Be mindful of the LLM's knowledge cutoff if asking for information about very recent movies not in its training data (though here we provide the movie details directly).
7.  **Fact-Checking (if LLM generates external info):** If the LLM were allowed to bring in external knowledge for explanations, a fact-checking layer might be needed. In our current setup, we are trying to constrain it to the provided data.

In [ ]:
async def full_conversational_pipeline():
    global synthetic_products_df, movie_info_map, recbole_model, recbole_dataset, recommendations, liked_movie_details_for_prompt, U_ITEM_FOUND

    # Re-check U_ITEM_FOUND at the start of the pipeline
    u_item_check_path = os.path.join(ML_100K_RAW_PATH, 'u.item')
    U_ITEM_FOUND = os.path.exists(u_item_check_path)
    if not U_ITEM_FOUND:
        print(f"CRITICAL (Pipeline Start): 'u.item' not found at {u_item_check_path}. Cannot proceed with conversational pipeline.")
        return

    # 1. Load Movie Info
    movie_info_map = load_movie_titles_and_genres(ML_100K_RAW_PATH)
    if not movie_info_map: return

    # 2. Load RecBole Model
    if MODEL_CHECKPOINT_PATH and os.path.exists(MODEL_CHECKPOINT_PATH):
        try:
            config, model, dataset, _, _, _ = load_data_and_model(model_file=MODEL_CHECKPOINT_PATH)
            recbole_model = model
            recbole_dataset = dataset
            recbole_model.eval()
            recbole_model.to(torch.device('cpu'))
            print(f"Successfully loaded model: {recbole_model.model_name} on device: {next(recbole_model.parameters()).device}")
        except Exception as e:
            print(f"Error loading RecBole model in full pipeline: {e}")
            recbole_model, recbole_dataset = None, None 
            return 
    else:
        print("Model checkpoint path not valid in full pipeline. Cannot proceed.")
        return

    # 3. Simulate User and Get Recommendations
    if recbole_model and recbole_dataset and movie_info_map:
        recommendations, liked_movie_details_for_prompt = get_recommendations_for_user(
            recbole_model, 
            recbole_dataset, 
            SIMULATED_USER_ID, 
            SIMULATED_LIKED_MOVIE_ORIGINAL_IDS, 
            NUM_RECOMMENDATIONS,
            movie_info_map
        )
        print(f"\n--- Simulated User {SIMULATED_USER_ID} (Full Pipeline) ---")
        print("Liked Movies:")
        if liked_movie_details_for_prompt: 
            for movie in liked_movie_details_for_prompt: print(f"- {movie['title']} (Genres: {movie['genres']})")
        else:
            print("No liked movie details to display for prompt context.")

        print("\nTop Recommendations:")
        if recommendations:
            for i, movie in enumerate(recommendations): print(f"{i+1}. {movie['title']} (Genres: {movie['genres']})")
        else:
            print("No recommendations generated in full pipeline.")
    else:
        print("Skipping recommendation generation in full pipeline due to missing components.")
        recommendations = [] 
        liked_movie_details_for_prompt = []


    # 4. Run Chatbot Example
    if SDK_CONFIGURED_SUCCESSFULLY and recommendations: 
        await run_chatbot_example()
    else:
        print("\nSkipping chatbot example in full pipeline due to missing SDK config or recommendations.")

In [ ]:
await full_conversational_pipeline()